In [1]:
# Updating pip and installing keras-tuner package
!/opt/conda/bin/python3.7 -m pip install --upgrade pip
!pip install -U keras-tuner

     |████████████████████████████████| 1.5 MB 2.9 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.1
    Uninstalling pip-20.1:
      Successfully uninstalled pip-20.1
     |████████████████████████████████| 54 kB 1.0 MB/s eta 0:00:011
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-py3-none-any.whl size=73198 sha256=7962a4980603548ed9c6715b5d1800e21ea323a0b20444950561e7ab22d3db44
  Stored in directory: /root/.cache/pip/wheels/0b/cf/2f/1a1749d3a3650fac3305a8d7f9237b6de7c41068e2f8520ca2
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=86cb18b0e31f677ca2dd2d0a792b837c0440a8af36333ccd3f9de6c0da802e69
  Stored in directory: /root/.cache/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner terminaltables


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from kerastuner import HyperModel, Objective
import tensorflow as tf
from kerastuner.tuners import RandomSearch
import keras.backend as K

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using TensorFlow backend.


/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [3]:
# Initializing the embedding dimention
batch_size = 64
embedding_dim = 512

# Read the input data.
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

# Fill all the na data with empty character.
train_df = train_df.fillna('empty')
test_df = test_df.fillna('empty')

In [4]:
# Join with the text, keyword as well as the location
train_df['text'] = train_df['text'] + ' ' + train_df['keyword'].astype(str) + ' ' + train_df['location'].astype(str)
test_df['text'] = test_df['text'] + ' ' + test_df['keyword'].astype(str) + ' ' + test_df['location'].astype(str)

# Strip off the whitespace from the front and back of the sentence
train_df['text'] = train_df['text'].str.strip()
test_df['text'] = test_df['text'].str.strip()

# Replace all the links, with just link as word.
# It matters more to know if there is a link or not in place of which link it is actually.
# But this remains to be seen later.
train_df['text'] = train_df['text'].str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'link')
test_df['text'] = test_df['text'].str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'link')

In [5]:
# To see the output of the pre-processing steps.
train_df.to_csv('train.csv', index=False)

# Drop the kexword and location column as it is not with the text column
train_df.drop(columns=['keyword', 'location'])

# Use the tokenizer and remove all the special characters. Add oov_character as irrelevant
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="'<irrelevant>'",
                                                  filters='!"$%&()*+.,-/:;=?@[\]^_`{|}~# ')

# Fit the tokenizer on the trainig data.
tokenizer.fit_on_texts(train_df.text)

In [6]:
# Convert the texts to tokens
train_df['text_tokenized'] = tokenizer.texts_to_sequences(train_df.text)
test_df['text_tokenized'] = tokenizer.texts_to_sequences(test_df.text)

# Pad the sequence as we will like to have sentences of equal length.
# We can also use bucket with sequence length.
np_matrix_train = tf.keras.preprocessing.sequence.pad_sequences(train_df['text_tokenized'])
np_matrix_train = np.append(np_matrix_train, np.expand_dims(train_df['target'], axis=-1), axis=1)

In [7]:
# convert the data to x and y, Later batch the dataset.
train_dataset = tf.data.Dataset.from_tensor_slices(np_matrix_train)
train_dataset_all = train_dataset.map(lambda x: (x[:-1], x[-1])).batch(batch_size)



# Do the similar to the test dataset.
np_matrix_test = tf.keras.preprocessing.sequence.pad_sequences(test_df['text_tokenized'])
test_dataset = tf.data.Dataset.from_tensor_slices(np_matrix_test).batch(1)

In [8]:
def is_test(x, y):
    return x % 5 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

validation_dataset = train_dataset_all.enumerate() \
                    .filter(is_test) \
                    .map(recover)

train_dataset = train_dataset_all.enumerate() \
                    .filter(is_train) \
                    .map(recover)

In [9]:
!rm -rf  ./real_or_not*

In [10]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
# Create the keras tuner model.
class MyHyperModel(HyperModel):
    
    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, embedding_dim))
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(tf.keras.layers.Conv1D(filters=hp.Choice('num_filters', values=[32, 64], default=64),activation='relu',
                                             kernel_size=3,
                                             bias_initializer='glorot_uniform'))
            model.add(tf.keras.layers.MaxPool1D())
        
        model.add(tf.keras.layers.GlobalMaxPool1D())
        
        for i in range(hp.Int('num_layers_rnn', 1, 3)):
            model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
            model.add(tf.keras.layers.Dropout(0.2))
        
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        
        model.compile(
            optimizer=hp.Choice('optimizer', values= ['Adam', 'Adadelta', 'Adamax']),
            loss='binary_crossentropy',
            metrics=[f1])
        return model


hypermodel = MyHyperModel()

tuner = RandomSearch(
    hypermodel,
    objective=Objective('val_f1', direction="max"),
    max_trials=15,
    directory='./',
    project_name='real_or_not')

tuner.search(train_dataset,
             epochs=10, validation_data=validation_dataset)


Epoch 1/10
95/95 [==============================].6941 - f1: 0.56 - 1s 616ms/step - loss: 0.6922 - f1: 0.644 - 1s 429ms/step - loss: 0.6952 - f1: 0.592 - 1s 336ms/step - loss: 0.6968 - f1: 0.545 - 1s 278ms/step - loss: 0.6967 - f1: 0.556 - 1s 239ms/step - loss: 0.6954 - f1: 0.585 - 1s 212ms/step - loss: 0.6970 - f1: 0.549 - 2s 192ms/step - loss: 0.6961 - f1: 0.577 - 2s 177ms/step - loss: 0.6981 - f1: 0.527 - 2s 165ms/step - loss: 0.6997 - f1: 0.487 - 2s 155ms/step - loss: 0.7006 - f1: 0.461 - 2s 147ms/step - loss: 0.7014 - f1: 0.438 - 2s 139ms/step - loss: 0.7017 - f1: 0.429 - 2s 132ms/step - loss: 0.7005 - f1: 0.451 - 2s 126ms/step - loss: 0.6995 - f1: 0.475 - 2s 121ms/step - loss: 0.6987 - f1: 0.490 - 2s 116ms/step - loss: 0.6980 - f1: 0.504 - 2s 112ms/step - loss: 0.6978 - f1: 0.510 - 2s 108ms/step - loss: 0.6978 - f1: 0.512 - 2s 104ms/step - loss: 0.6981 - f1: 0.508 - 2s 101ms/step - loss: 0.6982 - f1: 0.504 - 2s 99ms/step - loss: 0.6976 - f1: 0.515 - 2s 96ms/step - loss: 0.6975 - 

95/95 [==============================] - ETA: 7s - loss: 0.6995 - f1: 0.45 - ETA: 5s - loss: 0.6952 - f1: 0.51 - ETA: 4s - loss: 0.6949 - f1: 0.49 - ETA: 4s - loss: 0.6951 - f1: 0.49 - ETA: 4s - loss: 0.6954 - f1: 0.48 - ETA: 3s - loss: 0.6962 - f1: 0.43 - ETA: 3s - loss: 0.6966 - f1: 0.40 - ETA: 3s - loss: 0.6958 - f1: 0.43 - ETA: 3s - loss: 0.6954 - f1: 0.46 - ETA: 3s - loss: 0.6951 - f1: 0.47 - ETA: 3s - loss: 0.6953 - f1: 0.46 - ETA: 3s - loss: 0.6949 - f1: 0.47 - ETA: 2s - loss: 0.6953 - f1: 0.46 - ETA: 2s - loss: 0.6952 - f1: 0.46 - ETA: 2s - loss: 0.6952 - f1: 0.46 - ETA: 2s - loss: 0.6949 - f1: 0.47 - ETA: 2s - loss: 0.6949 - f1: 0.47 - ETA: 2s - loss: 0.6948 - f1: 0.48 - ETA: 2s - loss: 0.6946 - f1: 0.49 - ETA: 2s - loss: 0.6946 - f1: 0.49 - ETA: 2s - loss: 0.6946 - f1: 0.49 - ETA: 2s - loss: 0.6946 - f1: 0.49 - ETA: 2s - loss: 0.6946 - f1: 0.50 - ETA: 2s - loss: 0.6945 - f1: 0.50 - ETA: 2s - loss: 0.6945 - f1: 0.50 - ETA: 1s - loss: 0.6945 - f1: 0.50 - ETA: 1s - loss: 0.6945 

95/95 [==============================] - ETA: 8s - loss: 0.6908 - f1: 0.51 - ETA: 5s - loss: 0.6918 - f1: 0.54 - ETA: 4s - loss: 0.6918 - f1: 0.48 - ETA: 4s - loss: 0.6918 - f1: 0.46 - ETA: 3s - loss: 0.6922 - f1: 0.42 - ETA: 3s - loss: 0.6926 - f1: 0.39 - ETA: 3s - loss: 0.6926 - f1: 0.39 - ETA: 3s - loss: 0.6927 - f1: 0.38 - ETA: 3s - loss: 0.6929 - f1: 0.38 - ETA: 3s - loss: 0.6930 - f1: 0.40 - ETA: 3s - loss: 0.6930 - f1: 0.42 - ETA: 3s - loss: 0.6930 - f1: 0.41 - ETA: 3s - loss: 0.6928 - f1: 0.42 - ETA: 3s - loss: 0.6929 - f1: 0.41 - ETA: 2s - loss: 0.6928 - f1: 0.41 - ETA: 2s - loss: 0.6930 - f1: 0.41 - ETA: 2s - loss: 0.6929 - f1: 0.42 - ETA: 2s - loss: 0.6928 - f1: 0.42 - ETA: 2s - loss: 0.6927 - f1: 0.41 - ETA: 2s - loss: 0.6928 - f1: 0.42 - ETA: 2s - loss: 0.6928 - f1: 0.42 - ETA: 2s - loss: 0.6928 - f1: 0.42 - ETA: 2s - loss: 0.6928 - f1: 0.42 - ETA: 2s - loss: 0.6927 - f1: 0.42 - ETA: 2s - loss: 0.6927 - f1: 0.43 - ETA: 2s - loss: 0.6927 - f1: 0.43 - ETA: 2s - loss: 0.6928 

Epoch 1/10
95/95 [==============================]: 0.7048 - f1: 0.487 - 1s 371ms/step - loss: 0.6960 - f1: 0.628 - 1s 259ms/step - loss: 0.6981 - f1: 0.593 - 1s 204ms/step - loss: 0.7015 - f1: 0.538 - 1s 170ms/step - loss: 0.7004 - f1: 0.552 - 1s 147ms/step - loss: 0.6977 - f1: 0.591 - 1s 131ms/step - loss: 0.7003 - f1: 0.558 - 1s 119ms/step - loss: 0.6976 - f1: 0.588 - 1s 110ms/step - loss: 0.7008 - f1: 0.543 - 1s 102ms/step - loss: 0.7034 - f1: 0.501 - 1s 96ms/step - loss: 0.7050 - f1: 0.479 - 1s 91ms/step - loss: 0.7068 - f1: 0.45 - 1s 87ms/step - loss: 0.7079 - f1: 0.44 - 1s 83ms/step - loss: 0.7060 - f1: 0.46 - 1s 80ms/step - loss: 0.7037 - f1: 0.49 - 1s 77ms/step - loss: 0.7023 - f1: 0.51 - 1s 75ms/step - loss: 0.7009 - f1: 0.53 - 1s 73ms/step - loss: 0.7005 - f1: 0.53 - 1s 71ms/step - loss: 0.7003 - f1: 0.54 - 1s 69ms/step - loss: 0.7007 - f1: 0.53 - 1s 68ms/step - loss: 0.7013 - f1: 0.53 - 1s 66ms/step - loss: 0.7002 - f1: 0.54 - 1s 65ms/step - loss: 0.7002 - f1: 0.54 - 2s 64ms

In [ ]:
tuner.results_summary()

In [ ]:
models = tuner.get_best_models(num_models=1)

models[0].summary()

In [ ]:
models[0].predict(test_dataset)

In [ ]:
result_dataframe = pd.DataFrame(columns=['id', 'target'])
result_dataframe['id'] = test_df['id']
result_dataframe['target'] = np.where(np.array(models[0].predict(test_dataset)) >= 0.5, 1, 0 )
result_dataframe.to_csv('result.csv', index= False)